## Gift Contract Testing

In [1]:
from web3.auto import w3
import pandas as pd
from cyberpy import generate_wallet, address_to_address
from cyberpy._message_signer import Message
from cyberpy._wallet import seed_to_privkey
from eth_account.messages import encode_defunct
import json
import random
from time import time, sleep
import ipfshttpclient
from tqdm.notebook import tqdm
from base64 import b64encode
from multiprocess import Pool
from math import ceil
from dotenv import dotenv_values
from itertools import chain

from contract_utils import execute_bash, instantiate_contract, execute_contract_bash, query_contract, get_ipfs_cid_from_str, get_proofs, ContractUtils


ipfs_client = ipfshttpclient.connect()

NUMBER_OF_PARTICIPANTS = 20_000
NUMBER_OF_ACTIVATED_PARTICIPANTS = 5_000
INITIAL_BALANCE = str(30_000_000_000)
COEF_UP = str(13)
COEF_DOWN = str(7)
TARGET_CLAIM = str(2_000)
CLAIM_AMOUNT_LIST = [int(round(20_000 * random.random(), -3)) + 20_000 for _ in range(NUMBER_OF_PARTICIPANTS)]

NUMBER_OF_THREADS = 100
KEY_PHRASE = 'KEY PHRASE'
NICKNAME_LIST = [f'john{round(time())}{number}' for number in range(NUMBER_OF_PARTICIPANTS)]
AVATAR_CID_LIST = [get_ipfs_cid_from_str(_nickname + '_avatar') for _nickname in NICKNAME_LIST]
PREFIXES = ['cosmos', 'osmo', 'terra']

WALLET_ADDRESS = dotenv_values('.env')['WALLET_ADDRESS']
WALLET_SEED = dotenv_values('.env')['WALLET_SEED']
PYTHON_PATH = dotenv_values('.env')['PYTHON_PATH']

DISPLAY_TX_EXECUTION = False
CALCULATE_ADDRESS_SET = True
CALCULATE_PROOFS = True
SEND_COINS = True

INIT_SUBGRAPH_CONTRACTS = False
SUBGRAPH_CODE_ID = str(4)
NAME_SUBGRAPH_CONTRACT_ADDRESS = 'bostrom1zwv6feuzhy6a9wekh96cd57lsarmqlwxdypdsplw6zhfncqw6ftq67yzdk'
AVATAR_SUBGRAPH_CONTRACT_ADDRESS = 'bostrom1ghd753shjuwexxywmgs4xz7x2q732vcnkm6h2pyv9s6ah3hylvrqxkr9dg'
PROOF_SUBGRAPH_CONTRACT_ADDRESS = 'bostrom1xt4ahzz2x8hpkc0tk6ekte9x6crw4w6u0r67cyt3kz9syh24pd7s4t9edr'

INIT_PASSPORT_CONTRACT = False
PASSPORT_CODE_ID = str(7)
PASSPORT_CONTRACT_ADDRESS = 'bostrom1fzm6gzyccl8jvdv3qq6hp9vs6ylaruervs4m06c7k0ntzn2f8faq7ha2z2'

INIT_TREASURY_CONTRACT = False
TREASURY_CODE_ID = str(5)
TREASURY_CONTRACT_ADDRESS = 'bostrom19ttnpna4fpwuhw9dwjukah5esz9ruutquuvhxa7ulyyk047vvpvs7nh8wu'

INIT_GIFT_CONTRACT = False
GIFT_CODE_ID = str(2)
GIFT_CONTRACT_ADDRESS = 'bostrom1gc5wcdn9ges00w0l2cfxd7r2puyflak5dmkg26rsh083afmnrjxq3q6aaa'

PARTICIPANTS_FILE_NAME = 'temp/participants_test_data_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.csv'
ROOT_SOURCE_FILE_NAME = 'temp/root_testing_source_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.json'
PROOF_FILE_NAME = 'temp/proof_testing_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.json'
ROOT_FILE_NAME = 'temp/root_testing_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses'

/home/grovybear/.local/lib/python3.9/site-packages/ipfshttpclient/client/__init__.py:75: VersionMismatch: Unsupported daemon version '0.12.2' (not in range: 0.5.0 ≤ … < 0.9.0)
  warnings.warn(exceptions.VersionMismatch(version, minimum, maximum))


### Generate addresses and sign messages

In [2]:
def generate_wallet_data(i: int, prefixes: list = PREFIXES) -> dict:

    prefix = random.choice(prefixes)

    claim_item = {'amount': CLAIM_AMOUNT_LIST[i],
              'nickname': NICKNAME_LIST[i],
              'avatar': AVATAR_CID_LIST[i]}

    # Generate Ethereum wallet
    ethereum_wallet = w3.eth.account.create(KEY_PHRASE)
    claim_item['ethereum_address'] = ethereum_wallet.address.lower()
    claim_item['ethereum_private_key'] = ethereum_wallet.privateKey.hex()

    # Generate Bostrom and Cosmos wallet
    bostrom_wallet = generate_wallet()
    claim_item['bostrom_address'] = bostrom_wallet['address']
    claim_item['cosmos_address'] = address_to_address(bostrom_wallet['address'], prefix)
    claim_item['cosmos_seed'] = bostrom_wallet['seed']

    # Create message
    claim_item['message'] = f"{claim_item['bostrom_address']}:QmRX8qYgeZoYM3M5zzQaWEpVFdpin6FvVXvp6RPQK3oufV"

    # Sign message form Ethereum address
    ethereum_signed_message = \
        w3.eth.account.sign_message(
            signable_message=encode_defunct(text=claim_item['message']),
            private_key=ethereum_wallet.privateKey)
    claim_item['ethereum_message_signature'] = ethereum_signed_message.signature.hex()

    # Sign message form Cosmos address
    cosmos_msg = Message(privkey=seed_to_privkey(seed=claim_item['cosmos_seed']))
    cosmos_msg.add_message(signing_message=claim_item['message'], signer_prefix=prefix)
    claim_item['cosmos_message_signed_row'] = cosmos_msg.get_signed_message()
    claim_item['cosmos_message_signature'] = b64encode(json.dumps(claim_item['cosmos_message_signed_row']).replace('\n', '').replace(' ', '').encode('utf-8')).decode("utf-8")

    # Verify message
    assert claim_item['ethereum_address'] == w3.eth.account.recover_message(
        signable_message=encode_defunct(text=claim_item['message']),
        signature=claim_item['ethereum_message_signature']).lower()

    return claim_item


if CALCULATE_ADDRESS_SET:

    tasks = list(range(NUMBER_OF_PARTICIPANTS))
    print(f'Number of tasks: {len(tasks)}')
    print(f'Number of threads: {NUMBER_OF_THREADS}')
    with Pool(processes=NUMBER_OF_THREADS) as pool:
        claims_list = list(tqdm(pool.imap(generate_wallet_data, tasks), total=len(tasks)))

    claims_df = pd.DataFrame(claims_list)
    claims_df.to_csv('temp/claims_ethereum_test_data_without_proof_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.csv')
else:
    claims_df = pd.read_csv('temp/claims_ethereum_test_data_without_proof_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.csv')
claims_df.head()

,Unnamed: 0,amount,nickname,avatar,ethereum_address,ethereum_private_key,bostrom_address,cosmos_address,cosmos_seed,message,ethereum_message_signature,cosmos_message_signed_row,cosmos_message_signature
0,0,34000,john16548361750,QmaaXjnSDLGwpSRHdzUU8XrLjVJPt6E5pE5JavMyTCD381,0x3691db823a183aee18e998f2ddd77076a11fd74c,0x2aa29fbec6aed68bf55a7025c4280b85d38683edf00a...,bostrom14qx4d5nwyx5vj4e2gs6zfamjzydnvruydrj6gy,terra14qx4d5nwyx5vj4e2gs6zfamjzydnvruyg5uf5r,hope captain swallow label fish clarify demise...,bostrom14qx4d5nwyx5vj4e2gs6zfamjzydnvruydrj6gy...,0xb38a11a271602c2dc33c81602ad994810f9cf48dfb4d...,{'pub_key': 'AyjWt69QbKZ4V09ZCXKDN+bRgnw0H3R9B...,eyJwdWJfa2V5IjoiQXlqV3Q2OVFiS1o0VjA5WkNYS0ROK2...
1,1,34000,john16548361751,QmT7Qs7FRxLjjzJD5QxY7BDZ4R1VgdavE4NcNjQzr3EV1D,0xb42fe61f2d97178c18d6f36c637e55ec207a5441,0xe6df5403ea994063ff6e8ae25596433c157bf2cc1e97...,bostrom17wuryy4kexsgyq6rq2k3x0735n4ntqeq3flav5,osmo17wuryy4kexsgyq6rq2k3x0735n4ntqeq6pc7yp,science surprise lunar hip truth citizen excha...,bostrom17wuryy4kexsgyq6rq2k3x0735n4ntqeq3flav5...,0x84c8f25ca81c784d243e406ec8a4a1e389aa728dc235...,{'pub_key': 'A9bjehAjjY32eiYHqWier3WVEOdZUwZ8Q...,eyJwdWJfa2V5IjoiQTliamVoQWpqWTMyZWlZSHFXaWVyM1...
2,2,33000,john16548361752,QmQrf4uaZFS8YyouRuYW7RdSjZAY18jxB9t6NrWWqa9WKn,0x597ad4478d64d2702ad3187fd07ed2159a9b38d3,0x046508e879cf32970d3150c439b20dbf4b5aedeccbe6...,bostrom1ag0uyutudtdtr7aenpc0c0jaedrwrsjf0jfury,terra1ag0uyutudtdtr7aenpc0c0jaedrwrsjf2980lr,broccoli year exclude ship butter install goss...,bostrom1ag0uyutudtdtr7aenpc0c0jaedrwrsjf0jfury...,0xfc895e22ddcfc885b17b91fbc21146aa422d64e2ac43...,{'pub_key': 'AtTl2h3Nafoifz384IlN2M97SvoX/oZiQ...,eyJwdWJfa2V5IjoiQXRUbDJoM05hZm9pZnozODRJbE4yTT...
3,3,32000,john16548361753,QmYLg2fgKA6ptwXa4mFXyq2JqjEyTWMrFXwvu3J23zbepW,0xff0cf02f40e8021a91c48b79b7412c7b1c568b2f,0x17ab2a2fbc4dc59d3d837892de0af854d922e79117bd...,bostrom1gqz7pm32fcvzzskk7dg3kp0vgzs4aqfad5ulu4,cosmos1gqz7pm32fcvzzskk7dg3kp0vgzs4aqfaw8gvzj,autumn nominee slab attend iron odor antique w...,bostrom1gqz7pm32fcvzzskk7dg3kp0vgzs4aqfad5ulu4...,0x9a6179bbb941309bd5247e82a30f6b1fab4f63bcdc87...,{'pub_key': 'Amw0GvH82F8z30EaAGnRqT75oSpWxMWzI...,eyJwdWJfa2V5IjoiQW13MEd2SDgyRjh6MzBFYUFHblJxVD...
4,4,38000,john16548361754,QmNWX5GLuXzhhtZdxNAnWWC42gqJsA6A2fjsseB5HPVDyw,0xd8b950ffa0b84b4d73e377a5ca86ea0c233a83c7,0x66907d36d32bd3e99be73912172ebb6198a768dfa049...,bostrom1x9966p9dwtcju7vy6lcc4mvvqsj5rnffxsz79q,cosmos1x9966p9dwtcju7vy6lcc4mvvqsj5rnff9rkdm8,aware card grass item rural swallow alcohol co...,bostrom1x9966p9dwtcju7vy6lcc4mvvqsj5rnffxsz79q...,0x59f2175d252ba9a551c8f9e957e26592769e2181e594...,{'pub_key': 'AtDo/b+BravMbmqq8qoD40ROzNA84J2iY...,eyJwdWJfa2V5IjoiQXREby9iK0JyYXZNYm1xcThxb0Q0MF...


### Create Merkle Tree, Get Root and Proofs

In [3]:
root_source_list_ethereum = \
    [{'address': _item[0],
      'amount': str(_item[1])}
     for _item in claims_df.loc[:, ['ethereum_address', 'amount']].values.tolist()]
root_source_list_cosmos = \
    [{'address': _item[0],
      'amount': str(_item[1])}
     for _item in claims_df.loc[:, ['cosmos_address', 'amount']].values.tolist()]
root_source_list = list(chain.from_iterable(zip(root_source_list_ethereum, root_source_list_cosmos)))

with open(ROOT_SOURCE_FILE_NAME, 'w') as outfile:
    outfile.write(str(root_source_list).replace("'", '"'))

In [4]:
NUMBER_OF_THREADS = 30
BASH_SIZE = 2000

tasks = set(
    (
        (ROOT_SOURCE_FILE_NAME,
         f'temp/proofs_{i}.json',
         i * BASH_SIZE,
         min(NUMBER_OF_ACTIVATED_PARTICIPANTS * 2 + 1, (i + 1) * BASH_SIZE + 1))
        for i in range(ceil(2 * NUMBER_OF_ACTIVATED_PARTICIPANTS/BASH_SIZE))
    )
)
if CALCULATE_PROOFS:
    print(f'Number of addresses: {2 * NUMBER_OF_PARTICIPANTS:>,}\nNumber of activated participants: {NUMBER_OF_ACTIVATED_PARTICIPANTS:>,}\nNumber of threads: {NUMBER_OF_THREADS:>,}\nBash size: {BASH_SIZE:>,}\nNumber of tasks: {len(tasks):>,}')
    with Pool(processes=NUMBER_OF_THREADS) as pool:
        res = pool.starmap(get_proofs, tqdm(tasks, total=len(tasks)))
    assert res == [True] * len(res)

In [5]:
if CALCULATE_PROOFS:
    roots = []
    proofs_df = pd.DataFrame(columns=['address', 'amount', 'proof'])
    for task in tasks:
        with open(task[1], 'r') as proof_file:
            root_and_proof_json = json.load(proof_file)
        roots.append(root_and_proof_json['merkle_root'])
        proofs_df = pd.concat([proofs_df, pd.DataFrame(root_and_proof_json['proofs'])], ignore_index=True)
    assert roots == [roots[0]] * len(roots)
    root = roots[0]

    cosmos_proofs_df = proofs_df[~proofs_df.address.str.startswith('0x')][['address', 'proof']]
    ethereum_proofs_df = proofs_df[proofs_df.address.str.startswith('0x')][['address', 'proof']]

    claims_with_proofs_df = claims_df\
        .merge(
            ethereum_proofs_df.rename(columns={'address': 'ethereum_address', 'proof': 'ethereum_proof'}),
            how='inner',
            on='ethereum_address')\
        .merge(
            cosmos_proofs_df.rename(columns={'address': 'cosmos_address', 'proof': 'cosmos_proof'}),
            how='inner',
            on='cosmos_address')

    claims_with_proofs_df.to_csv(PARTICIPANTS_FILE_NAME)
else:
    claims_with_proofs_df = pd.read_csv(PARTICIPANTS_FILE_NAME)
    claims_with_proofs_df.loc[:, 'ethereum_proof'] = claims_with_proofs_df['ethereum_proof'].map(lambda x: x.replace('\'', '').replace('[', '').replace(']', '').split(', '))
    claims_with_proofs_df.loc[:, 'cosmos_proof'] = claims_with_proofs_df['cosmos_proof'].map(lambda x: x.replace('\'', '').replace('[', '').replace(']', '').split(', '))
    with open('temp/proofs_1.json', 'r') as proof_file:
        root_and_proof_json = json.load(proof_file)
    root = root_and_proof_json['merkle_root']
claims_with_proofs_df.head()

,Unnamed: 0,amount,nickname,avatar,ethereum_address,ethereum_private_key,bostrom_address,cosmos_address,cosmos_seed,message,ethereum_message_signature,cosmos_message_signed_row,cosmos_message_signature,ethereum_proof,cosmos_proof
0,0,34000,john16548361750,QmaaXjnSDLGwpSRHdzUU8XrLjVJPt6E5pE5JavMyTCD381,0x3691db823a183aee18e998f2ddd77076a11fd74c,0x2aa29fbec6aed68bf55a7025c4280b85d38683edf00a...,bostrom14qx4d5nwyx5vj4e2gs6zfamjzydnvruydrj6gy,terra14qx4d5nwyx5vj4e2gs6zfamjzydnvruyg5uf5r,hope captain swallow label fish clarify demise...,bostrom14qx4d5nwyx5vj4e2gs6zfamjzydnvruydrj6gy...,0xb38a11a271602c2dc33c81602ad994810f9cf48dfb4d...,{'pub_key': 'AyjWt69QbKZ4V09ZCXKDN+bRgnw0H3R9B...,eyJwdWJfa2V5IjoiQXlqV3Q2OVFiS1o0VjA5WkNYS0ROK2...,[4710b3ef8cb686c8b316654c7a6587fe33c256755bd65...,[172e89389492e7389a96c98cae5190ffcae04a8d2eb85...
1,1,34000,john16548361751,QmT7Qs7FRxLjjzJD5QxY7BDZ4R1VgdavE4NcNjQzr3EV1D,0xb42fe61f2d97178c18d6f36c637e55ec207a5441,0xe6df5403ea994063ff6e8ae25596433c157bf2cc1e97...,bostrom17wuryy4kexsgyq6rq2k3x0735n4ntqeq3flav5,osmo17wuryy4kexsgyq6rq2k3x0735n4ntqeq6pc7yp,science surprise lunar hip truth citizen excha...,bostrom17wuryy4kexsgyq6rq2k3x0735n4ntqeq3flav5...,0x84c8f25ca81c784d243e406ec8a4a1e389aa728dc235...,{'pub_key': 'A9bjehAjjY32eiYHqWier3WVEOdZUwZ8Q...,eyJwdWJfa2V5IjoiQTliamVoQWpqWTMyZWlZSHFXaWVyM1...,[cc3c069b0ce8d95d69e48a7b4f3728d892a36270e3e52...,[5398e38c29e181c2ff18cd4df1215ca90677cca24eeb6...
2,2,33000,john16548361752,QmQrf4uaZFS8YyouRuYW7RdSjZAY18jxB9t6NrWWqa9WKn,0x597ad4478d64d2702ad3187fd07ed2159a9b38d3,0x046508e879cf32970d3150c439b20dbf4b5aedeccbe6...,bostrom1ag0uyutudtdtr7aenpc0c0jaedrwrsjf0jfury,terra1ag0uyutudtdtr7aenpc0c0jaedrwrsjf2980lr,broccoli year exclude ship butter install goss...,bostrom1ag0uyutudtdtr7aenpc0c0jaedrwrsjf0jfury...,0xfc895e22ddcfc885b17b91fbc21146aa422d64e2ac43...,{'pub_key': 'AtTl2h3Nafoifz384IlN2M97SvoX/oZiQ...,eyJwdWJfa2V5IjoiQXRUbDJoM05hZm9pZnozODRJbE4yTT...,[50a1a2537255c5e1f077442076b2273671acc7b2d5a46...,[ad1576871f19a385d00e90552c06a38d084ba80aa043f...
3,3,32000,john16548361753,QmYLg2fgKA6ptwXa4mFXyq2JqjEyTWMrFXwvu3J23zbepW,0xff0cf02f40e8021a91c48b79b7412c7b1c568b2f,0x17ab2a2fbc4dc59d3d837892de0af854d922e79117bd...,bostrom1gqz7pm32fcvzzskk7dg3kp0vgzs4aqfad5ulu4,cosmos1gqz7pm32fcvzzskk7dg3kp0vgzs4aqfaw8gvzj,autumn nominee slab attend iron odor antique w...,bostrom1gqz7pm32fcvzzskk7dg3kp0vgzs4aqfad5ulu4...,0x9a6179bbb941309bd5247e82a30f6b1fab4f63bcdc87...,{'pub_key': 'Amw0GvH82F8z30EaAGnRqT75oSpWxMWzI...,eyJwdWJfa2V5IjoiQW13MEd2SDgyRjh6MzBFYUFHblJxVD...,[e069f04ee7b66a3181a7fcc473cde8fb80b10ef8077f3...,[0da1f0fe7351add378fb84ade27270a297bce88360b95...
4,4,38000,john16548361754,QmNWX5GLuXzhhtZdxNAnWWC42gqJsA6A2fjsseB5HPVDyw,0xd8b950ffa0b84b4d73e377a5ca86ea0c233a83c7,0x66907d36d32bd3e99be73912172ebb6198a768dfa049...,bostrom1x9966p9dwtcju7vy6lcc4mvvqsj5rnffxsz79q,cosmos1x9966p9dwtcju7vy6lcc4mvvqsj5rnff9rkdm8,aware card grass item rural swallow alcohol co...,bostrom1x9966p9dwtcju7vy6lcc4mvvqsj5rnffxsz79q...,0x59f2175d252ba9a551c8f9e957e26592769e2181e594...,{'pub_key': 'AtDo/b+BravMbmqq8qoD40ROzNA84J2iY...,eyJwdWJfa2V5IjoiQXREby9iK0JyYXZNYm1xcThxb0Q0MF...,[5239d46e245ce4af1f69ec17f7786c6042fea7f62a0d8...,[9bb1d32faec7ad21dd219cce62196c374806fd1d77d36...


In [6]:
for i in range(ceil(NUMBER_OF_ACTIVATED_PARTICIPANTS/10_000)):
    claims_with_proofs_df.loc[i*10_000:(i+1)*10_000,:].to_csv(f'temp/{i}_{PARTICIPANTS_FILE_NAME.split("/")[1]}')

## Instantiate Contracts
### Instantiate SUBGRAPH Contracts

In [7]:
if INIT_SUBGRAPH_CONTRACTS:
    name_subgraph_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBGRAPH_CODE_ID,
            contract_label='test name subgraph')
    avatar_subgraph_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBGRAPH_CODE_ID,
            contract_label='test avatar subgraph',
            from_address=WALLET_ADDRESS)
    proof_subgraph_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBGRAPH_CODE_ID,
            contract_label='test proof subgraph',
            from_address=WALLET_ADDRESS)
else:
    name_subgraph_contract_address = NAME_SUBGRAPH_CONTRACT_ADDRESS
    avatar_subgraph_contract_address = AVATAR_SUBGRAPH_CONTRACT_ADDRESS
    proof_subgraph_contract_address = PROOF_SUBGRAPH_CONTRACT_ADDRESS
print(f'Name subgraph contract address: {name_subgraph_contract_address}\n'
      f'Avatar subgraph contract address: {avatar_subgraph_contract_address}\n'
      f'Proof subgraph contract address: {proof_subgraph_contract_address}')

Name subgraph contract address: bostrom1zwv6feuzhy6a9wekh96cd57lsarmqlwxdypdsplw6zhfncqw6ftq67yzdk
Avatar subgraph contract address: bostrom1ghd753shjuwexxywmgs4xz7x2q732vcnkm6h2pyv9s6ah3hylvrqxkr9dg
Proof subgraph contract address: bostrom1xt4ahzz2x8hpkc0tk6ekte9x6crw4w6u0r67cyt3kz9syh24pd7s4t9edr


### Instantiate Passport Contract

In [8]:
if INIT_PASSPORT_CONTRACT:
    passport_contract_address = \
        instantiate_contract(
            init_query=f'''{{"name":"CPT", "minter":"{WALLET_ADDRESS}", "owner":"{WALLET_ADDRESS}", "symbol":"CPT", "avatar_subgraph": "{avatar_subgraph_contract_address}", "name_subgraph": "{name_subgraph_contract_address}", "proof_subgraph": "{proof_subgraph_contract_address}"}}''',
            contract_code_id=PASSPORT_CODE_ID,
            contract_label='test passport',
            from_address=WALLET_ADDRESS)
else:
    passport_contract_address = PASSPORT_CONTRACT_ADDRESS
print(f'Passport contract address: {passport_contract_address}')

Passport contract address: bostrom1fzm6gzyccl8jvdv3qq6hp9vs6ylaruervs4m06c7k0ntzn2f8faq7ha2z2


### Set executor in the Subgraph Contracts

In [9]:
def set_executor_subgraph(subgraph_contract_address: str, new_executor_address: str, display_data: bool = False):
    return execute_contract_bash(execute_query=f'''{{"update_executer":{{"new_executer":"{new_executor_address}"}}}}''',
                                 contract_address=subgraph_contract_address,
                                 gas=600000,
                                 display_data=display_data)

if INIT_PASSPORT_CONTRACT or INIT_SUBGRAPH_CONTRACTS:
    set_executor_subgraph(subgraph_contract_address=name_subgraph_contract_address, new_executor_address=passport_contract_address)
    set_executor_subgraph(subgraph_contract_address=avatar_subgraph_contract_address, new_executor_address=passport_contract_address)
    set_executor_subgraph(subgraph_contract_address=proof_subgraph_contract_address, new_executor_address=passport_contract_address)

### Instantiate Treasury Contract

In [10]:
if INIT_TREASURY_CONTRACT:
    treasury_contract_address = \
        instantiate_contract(
            init_query=f'''{{"admins": ["{WALLET_ADDRESS}"], "mutable": true}}''',
            contract_code_id=TREASURY_CODE_ID,
            amount=INITIAL_BALANCE,
            contract_label='test treasury',
            from_address=WALLET_ADDRESS,
            display_data=False)
else:
    treasury_contract_address = TREASURY_CONTRACT_ADDRESS
print(f'Treasury contract address: {treasury_contract_address}')

Treasury contract address: bostrom19ttnpna4fpwuhw9dwjukah5esz9ruutquuvhxa7ulyyk047vvpvs7nh8wu


### Instantiate Gift Contract

In [11]:
if INIT_GIFT_CONTRACT:
    gift_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "passport":"{passport_contract_address}", "treasury":"{treasury_contract_address}", "allowed_native":"boot", "initial_balance":"{INITIAL_BALANCE}", "coefficient_up":"{COEF_UP}", "coefficient_down":"{COEF_DOWN}", "coefficient":"{COEF_UP}", "target_claim":"{TARGET_CLAIM}"}}''',
            contract_code_id=GIFT_CODE_ID,
            contract_label='test gift',
            from_address=WALLET_ADDRESS,
            display_data=False)
else:
    gift_contract_address = GIFT_CONTRACT_ADDRESS
print(f'Gift contract address: {gift_contract_address}')

Gift contract address: bostrom1gc5wcdn9ges00w0l2cfxd7r2puyflak5dmkg26rsh083afmnrjxq3q6aaa


### Initiate Class of Output Parsing

In [12]:
contract_utils = ContractUtils(ipfs_client=ipfs_client,
                               address_dict= {
                                   gift_contract_address: 'Gift Contract',
                                   passport_contract_address: 'Passport Contract',
                                   treasury_contract_address: 'Treasury Contract',
                                   WALLET_ADDRESS: 'Passport Owner Address',
                                   name_subgraph_contract_address: 'Name Subgraph Contract',
                                   avatar_subgraph_contract_address: 'Avatar Subgraph Contract',
                                   proof_subgraph_contract_address: 'Proof Subgraph Contract'})

### Set Allowance in Treasury contract

In [13]:
contract_utils.parse_contract_execution_json(
    contract_utils.send_coins(
        from_seed=WALLET_SEED,
        to_addresses=[treasury_contract_address],
        amounts=[INITIAL_BALANCE],
        gas=min(100_000 + 25_000, int(20e6)),
        display_data=DISPLAY_TX_EXECUTION))


Events

coin received
	receiver: Treasury Contract
	amount: 30000000000boot

coin spent
	spender: Passport Owner Address
	amount: 30000000000boot

message from bank bostrom1mxdtr8lruutugqtxgpw2sf2tl2mhzlq5fd2du0 /cosmos.bank.v1beta1.MsgMultiSend

transfer
	recipient: Treasury Contract
	amount: 30000000000boot
Gas used: 86,568
Tx hash: 823B7CB39E0B68EDD6ABC409CF595F9923BE936A7E7A386996185C83981C081E


In [14]:
def set_allowance(amount: str = INITIAL_BALANCE, display_data: bool = False):
    return contract_utils.execute_contract(
        execute_msg={"increase_allowance": {"spender": gift_contract_address, "amount": {"amount": amount, "denom": 'boot'}}},
        contract_address=treasury_contract_address,
        mnemonic=WALLET_SEED,
        gas=600000,
        display_data=display_data)

contract_utils.parse_contract_execution_json(
    set_allowance(display_data=False))


Events

execute
	execute contract: Treasury Contract

message from Passport Owner Address wasm /cosmwasm.wasm.v1.MsgExecuteContract

wasm
	_contract_address: Treasury Contract
	action: increase_allowance
	owner: Passport Owner Address
	spender: Gift Contract
	denomination: boot
	amount: 30,000,000,000
Gas used: 135,339
Tx hash: B680702CE0A4CBF4AA2E2A2CFA0221F6D30DC067528965E94476F1C9137EF913


### Register Merkle Root

In [15]:
root_register_output = contract_utils.execute_contract(
    execute_msg={"register_merkle_root": {"merkle_root": root}},
    mnemonic=WALLET_SEED,
    contract_address=gift_contract_address)
contract_utils.parse_contract_execution_json(root_register_output)


Events

execute
	execute contract: Gift Contract

message from Passport Owner Address wasm /cosmwasm.wasm.v1.MsgExecuteContract

wasm
	_contract_address: Gift Contract
	action: register_merkle_root
	merkle_root: 0cd70e3c34ff111de47721ca40ece8fad8d49549bfc16d29f5d074781bdd3ff1
Gas used: 130,976
Tx hash: F20198A196F13F8D1FB93E66116A66ADE77C7E43726AA8BEBAF419ADBD316E82


#### Get Merkle Root form the Gift Contract

In [16]:
print(f'Gift contract {gift_contract_address}')
print(query_contract(query='''{"merkle_root": {}}''',
                     contract_address=gift_contract_address))

Gift contract bostrom1gc5wcdn9ges00w0l2cfxd7r2puyflak5dmkg26rsh083afmnrjxq3q6aaa
{'data': {'merkle_root': '0cd70e3c34ff111de47721ca40ece8fad8d49549bfc16d29f5d074781bdd3ff1'}}


### Send coins to new addresses

In [17]:
NUMBER_ADDRESSES_IN_SENDING_CHUNK = 800

bostrom_addresses = claims_with_proofs_df.bostrom_address.to_list()
bostrom_addresses_chunks = [bostrom_addresses[i: i+ NUMBER_ADDRESSES_IN_SENDING_CHUNK] for i in range(0, len(bostrom_addresses), NUMBER_ADDRESSES_IN_SENDING_CHUNK)]

send_output = []
for i, bostrom_addresses_item in tqdm(enumerate(bostrom_addresses_chunks)):
    try:
        send_output.append(
            contract_utils.send_coins(
                from_seed=WALLET_SEED,
                to_addresses=bostrom_addresses_item,
                amounts=[1] * len(bostrom_addresses_item),
                gas=min(150_000 + 30_000 * len(bostrom_addresses_item), int(22e6)),
                display_data=DISPLAY_TX_EXECUTION))
    except Exception as e:
        print(f'Error in the {i} chunk: {e}')
        sleep(15)

0it [00:00, ?it/s]

In [ ]:
contract_utils.parse_contract_execution_json(contract_execution_json=send_output[0])

### Create Passports, Proof Addresses and Claim Gift

In [ ]:
def execute_participation(row_index: int) -> bool:
    _output, _error = execute_bash(bash_command=f'{PYTHON_PATH} create_passport_and_claim_job.py temp/{row_index//10_000}_{PARTICIPANTS_FILE_NAME.split("/")[1]} {row_index} {gift_contract_address}')
    if _output:
        return _output
    else:
        return _error

NUMBER_OF_THREADS = 100

tasks = list(range(NUMBER_OF_ACTIVATED_PARTICIPANTS))
print(f'Number of tasks: {len(tasks):>,}')
print(f'Number of threads: {NUMBER_OF_THREADS:>,}')


with Pool(processes=NUMBER_OF_THREADS) as pool:
    res_participation = list(tqdm(pool.imap(execute_participation, tasks), total=len(tasks)))

In [ ]:
import json
with open('temp/contract_participation_execution_log_2.txt', 'r') as f:
    data = json.load(f)
data

In [ ]:
contract_utils.parse_contract_execution_json(data['create'], row=claims_with_proofs_df.loc[1,:])

In [ ]:
contract_utils.parse_contract_execution_json(data['proof_ethereum'], row=claims_with_proofs_df.loc[1,:])

In [ ]:
contract_utils.parse_contract_execution_json(data['proof_cosmos'], row=claims_with_proofs_df.loc[1,:])

In [ ]:
contract_utils.parse_contract_execution_json(data['claim_cosmos'], row=claims_with_proofs_df.loc[1,:])

### Release Gift

In [20]:
def execute_release(row_index: int) -> bool:
    _output, _error = execute_bash(bash_command=f'{PYTHON_PATH} create_passport_and_claim_job.py temp/{row_index//10_000}_{PARTICIPANTS_FILE_NAME.split("/")[1]} {row_index} {gift_contract_address} True')
    if _output:
        return _output
    else:
        return _error


NUMBER_OF_THREADS = 100

tasks = list(range(NUMBER_OF_ACTIVATED_PARTICIPANTS))
print(f'Number of tasks: {len(tasks):>,}')
print(f'Number of threads: {NUMBER_OF_THREADS:>,}')


with Pool(processes=NUMBER_OF_THREADS) as pool:
    res_release = list(tqdm(pool.imap(execute_release, tasks), total=len(tasks)))

Number of tasks: 5,000
Number of threads: 100


  0%|          | 0/5000 [00:00<?, ?it/s]

In [ ]:
with open('temp/contract_release_execution_log_1.txt', 'r') as f:
    release_data = json.load(f)

In [ ]:
release_data

In [ ]:
contract_utils.parse_contract_execution_json(release_data['release_ethereum'], row=claims_with_proofs_df.loc[1,:])

In [ ]:
contract_utils.parse_contract_execution_json(release_data['release_cosmos'], row=claims_with_proofs_df.loc[1,:])

## Passport NFT testing

In [ ]:
def get_passport_id(bostrom_address: str) -> str:
    try:
        return query_contract(query=f'''{{"tokens": {{"owner": "{bostrom_address}"}}}}''',
                              contract_address=passport_contract_address)['data']['tokens'][0]
    except (json.JSONDecodeError, IndexError) as e:
        print(f'Error in the get passport: {e}')
        return ''

In [ ]:
def execute_passport_test(row_index: int, contract_utils=contract_utils) -> bool:
    row = claims_with_proofs_df.iloc[row_index]
    if row_index % 5 == 1:
        transfer_passport_json = contract_utils.transfer_passport(
            row,
            token_id=get_passport_id(row['bostrom_address']),
            to_address=WALLET_ADDRESS,
            display_data=DISPLAY_TX_EXECUTION)
        if row_index < 6:
            print('\nTRANSFER')
            contract_utils.parse_contract_execution_json(transfer_passport_json, row=row)
    elif row_index % 5 == 2:
        burn_passport_json = contract_utils.burn_passport(
            row,
            token_id=get_passport_id(row['bostrom_address']),
            display_data=DISPLAY_TX_EXECUTION)
        if row_index < 6:
            print('\nBURN')
            contract_utils.parse_contract_execution_json(burn_passport_json, row=row)
    elif row_index % 5 == 3:
        update_name_json = contract_utils.update_name(
            row,
            new_nickname=row['nickname'] + '_new',
            display_data=DISPLAY_TX_EXECUTION)
        if row_index < 6:
            print('\nUPDATE NAME')
            contract_utils.parse_contract_execution_json(update_name_json, row=row)
    elif row_index % 5 == 4:
        update_avatar_json = contract_utils.update_avatar(
            row,
            new_avatar=get_ipfs_cid_from_str(row['nickname'] + '_new_avatar'),
            display_data=DISPLAY_TX_EXECUTION)
        if row_index < 6:
            print('\nUPDATE AVATAR')
            contract_utils.parse_contract_execution_json(update_avatar_json, row=row)
    elif row_index % 5 == 0:
       remove_address_json = contract_utils.remove_address(
            row,
            removed_address=row['cosmos_address'],
            display_data=DISPLAY_TX_EXECUTION)
       if row_index < 6:
           print('\nREMOVE ADDRESS')
           contract_utils.parse_contract_execution_json(remove_address_json, row=row)
    print(f'Finish: {row_index}')
    return True

NUMBER_OF_THREADS = 100

tasks = list(range(NUMBER_OF_ACTIVATED_PARTICIPANTS))[:20]
print(f'Number of tasks: {len(tasks):>,}')
print(f'Number of threads: {NUMBER_OF_THREADS:>,}')

for row_index in tqdm(range(34, NUMBER_OF_ACTIVATED_PARTICIPANTS)):
    execute_passport_test(row_index)
# with Pool(processes=NUMBER_OF_THREADS) as pool:
#     # res_passport_test = list(tqdm(pool.imap(execute_passport_test, tasks), total=len(tasks)))
#     res_passport_test = list(pool.imap(execute_passport_test, tasks))

In [ ]:
# claims_with_proofs_df
execute_passport_test(0)

### Add tests for Passport contract
#### create_passport +
#### update_name +
#### update_avatar +
#### proof_address +
#### remove_address +
#### set_minter
#### set_owner
#### set_active
#### set_subgraphs +
#### transfer_nft +
#### send_nft
#### mint
#### burn +
#### approve
#### approve_all
#### revoke
#### revoke_all
#### Expirations
##### at_height
##### at_time
##### never

### Add tests for Gift contract
#### update_owner
#### update_passport_addr
#### update_target
#### register_merkle_root +
#### claim +
#### release +

### Add tests for subgraph contract
#### update_owner
#### update_executer +